# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 52 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2303.13172


extracting tarball to tmp_2303.13172...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:389: LatexWarning: Latex injecting: 'JWST_23385_230323.bbl' from 'tmp_2303.13172/JWST_23385_230323.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

extracting tarball to tmp_2303.13445...

 done.
Retrieving document from  https://arxiv.org/e-print/2303.13469


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2303.13445/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:389: LatexWarning: Latex injecting: 'authors.tex' from 'tmp_2303.13445/authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


extracting tarball to tmp_2303.13469...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.13172-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.13172) | **JOYS: JWST Observations of Young protoStars: Outflows and accretion in  the high-mass star-forming region IRAS23385+605**  |
|| <mark>H. Beuther</mark>, et al. -- incl., <mark>G. Perotti</mark>, <mark>M. Guedel</mark> |
|*Appeared on*| *2023-03-24*|
|*Comments*| *12 pages, 9 figures, accepted for Astronomy & Astrophysics, the paper is also available at this https URL*|
|**Abstract**| Aims: The JWST program JOYS (JWST Observations of Young protoStars) aims at characterizing the physical and chemical properties of young high- and low-mass star-forming regions, in particular the unique mid-infrared diagnostics of the warmer gas and solid-state components. We present early results from the high-mass star formation region IRAS23385+6053. Methods: The JOYS program uses the MIRI MRS with its IFU to investigate a sample of high- and low-mass star-forming protostellar systems. Results: The 5 to 28mum MIRI spectrum of IRAS23385+6053 shows a plethora of features. While the general spectrum is typical for an embedded protostar, we see many atomic and molecular gas lines boosted by the higher spectral resolution and sensitivity compared to previous space missions. Furthermore, ice and dust absorption features are also present. Here, we focus on the continuum emission, outflow tracers like the H2, [FeII] and [NeII] lines as well as the potential accretion tracer Humphreys alpha HI(7--6). The short-wavelength MIRI data resolve two continuum sources A and B, where mid-infrared source A is associated with the main mm continuum peak. The combination of mid-infrared and mm data reveals a young cluster in its making. Combining the mid-infrared outflow tracer H2, [FeII] and [NeII] with mm SiO data shows a complex interplay of at least three molecular outflows driven by protostars in the forming cluster. Furthermore, the Humphreys alpha line is detected at a 3-4sigma level towards the mid-infrared sources A and B. Following Rigliaco et al. (2015), one can roughly estimate accretion luminosities and corresponding accretion rates between ~2.6x10^-6 and ~0.9x10^-4 M_sun/yr. This is discussed in the context of the observed outflow rates. Conclusions: The analysis of the MIRI MRS observations for this young high-mass star-forming region reveals connected outflow and accretion signatures. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.13469-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.13469) | **JWST MIRI flight performance: The Medium-Resolution Spectrometer**  |
|| I. Argyriou, et al. -- incl., <mark>M. Wells</mark> |
|*Appeared on*| *2023-03-24*|
|*Comments*| *15 pages, 21 figures*|
|**Abstract**| The Medium-Resolution Spectrometer (MRS) provides one of the four operating modes of the Mid-Infrared Instrument (MIRI) on board the James Webb Space Telescope (JWST). The MRS is an integral field spectrometer, measuring the spatial and spectral distributions of light across the 5-28 $\mu m$ wavelength range with a spectral resolving power between 3700-1300. We present the MRS' optical, spectral, and spectro-photometric performance, as achieved in flight, and report on the effects that limit the instrument's ultimate sensitivity. The MRS flight performance has been quantified using observations of stars, planetary nebulae, and planets in our Solar System. The precision and accuracy of this calibration is checked against celestial calibrators with well known flux levels and spectral features. We find that the MRS geometric calibration has a distortion solution accuracy relative to the commanded position of 8 mas at 5 $\mu m$ and 23 mas at 28 $\mu m$. The wavelength calibration is accurate to within 9 km/sec at 5 $\mu m$ and 27 km/sec at 28 $\mu m$. The uncertainty in the absolute spectro-photometric calibration accuracy is estimated at 5.6 +- 0.7 %. The MIRI calibration pipeline is able to suppress the amplitude of spectral fringes to below 1.5 % for both extended and point sources across the entire wavelength range. The MRS PSF is 60 % broader than the diffraction limit along its long axis at 5 $\mu m$ and is 15 % broader at 28 $\mu m$. The MRS flight performance is found to be better than pre-launch expectations. The MRS is one of the most subscribed observing modes of JWST and is yielding many high-profile publications. It is currently humanity's most powerful instrument for measuring the mid-infrared spectra of celestial sources and is expected to continue as such for many years to come. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.13202-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.13202) | **Tentative detection of titanium oxide in the atmosphere of WASP-69 b  with a 4m ground-based telescope**  |
|| Q. Ouyang, et al. -- incl., <mark>M. Zhai</mark> |
|*Appeared on*| *2023-03-24*|
|*Comments*| *21 pages, 17 figures, 9 tables, Accepted for publication in MNRAS*|
|**Abstract**| Transiting planets provide unique opportunities for the atmospheric characterization of exoplanets as they can reveal composition and the temperature structures at the day-night terminator regions in planetary atmospheres, and help understand the atmospheric process and formation environments of exoplanets. Here, we present the optical transmission spectroscopic study of an inflated Saturn-mass planet WASP-69 b, obtained by the 4-meter ground-based telescope Southern Astrophysical Research Telescope (SOAR). We obtain spectroscopic transit light curves in 20 passbands from 502 to 890 nm, and fit them using Gaussian Processes and an analytical transit model to obtain independent transit depths for each. The derived transmission spectrum of WASP-69 b shows a slope with absorption depth increasing towards blue wavelengths, indicating a Rayleigh scattering in the atmosphere consistent with previous works. The retrieval analysis yields a tentative detection of TiO absorption feature in the transmission spectrum. We present the first results from the SOAR telescope to characterize exoplanetary atmospheres proving its capability and precision for hot Jupiters around bright stars in an area dominated by results from large ground-based telescopes or space telescopes. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.13445-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.13445) | **Synchronous rotation in the (136199) Eris-Dysnomia system**  |
|| G. M. Bernstein, et al. -- incl., <mark>M. Smith</mark> |
|*Appeared on*| *2023-03-24*|
|*Comments*| *Submitted to Planetary Science Journal*|
|**Abstract**| We combine photometry of Eris from a 6-month campaign on the Palomar 60-inch telescope in 2015, a 1-month Hubble Space Telescope WFC3 campaign in 2018, and Dark Energy Survey data spanning 2013--2018 to determine a light curve of definitive period $15.771\pm 0.008$~days (1-$\sigma$ formal uncertainties), with nearly sinusoidal shape and peak-to-peak flux variation of 3\%. This is consistent at part-per-thousand precision with the $P=15.78590\pm0.00005$~day period of Dysnomia's orbit around Eris, strengthening the recent detection of synchronous rotation of Eris by Szakats et al (2022) with independent data. Photometry from Gaia is consistent with the same light curve. We detect a slope of $0.05\pm0.01$~mag per degree of Eris' brightness with respect to illumination phase, intermediate between Pluto's and Charon's values. Variations of $0.3$~mag are detected in Dysnomia's brightness, plausibly consistent with a double-peaked light curve at the synchronous period. The synchronous rotation of Eris is consistent with simple tidal models initiated with a giant-impact origin of the binary, but is difficult to reconcile with gravitational capture of Dysnomia by Eris. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2303.13172.md
    + _build/html/tmp_2303.13172/./outflows.png
    + _build/html/tmp_2303.13172/./outflows.png
    + _build/html/tmp_2303.13172/./23385_overview2.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# JOYS: JWST Observations of Young protoStars

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.13172-b31b1b.svg)](https://arxiv.org/abs/2303.13172)<mark>Appeared on: 2023-03-24</mark> - _12 pages, 9 figures, accepted for Astronomy & Astrophysics, the paper is also available at this https URL_

</div>
<div id="authors">

<mark>H. Beuther</mark>, et al. -- incl., <mark>G. Perotti</mark>, <mark>M. Guedel</mark>

</div>
<div id="abstract">

**Abstract:** Understanding the earliest stages of star formation, and setting that into context with the general cycle of matter in the interstellar medium, is a central aspect of research with the James Webb Space Telescope (JWST). The JWST program JOYS (JWST Observations of Young protoStars) aims at characterizing the physical and chemical properties of young high- and low-mass star-forming regions, in particular the unique mid-infrared diagnostics of the warmer gas and solid-state components. We present early results from the high-mass star formation region IRAS 23385+6053. The JOYS program uses the Mid-Infrared Instrument (MIRI) Medium Resolution Spectrometer (MRS) with its Integral Field Unit (IFU) to investigate a sample of high- and low-mass star-forming protostellar systems. The full 5 to 28 $\mu$ m MIRI MRS spectrum of IRAS 23385+6053 shows a plethora of interesting features. While the general spectrum is typical for an embedded protostar, we see many atomic and molecular gas lines boosted by the higher spectral resolution and sensitivity compared to previous space missions. Furthermore, ice and dust absorption features are also present. Here, we focus on the continuum emission, outflow tracers like the $H_2$ (0--0)S(7), [ FeII ] ( $^4F_{9/2}$ -- $^6D_{9/2}$ ) and [ NeII ] ( $^2P_{1/2}-^2P_{3/2}$ ) lines as well as the potential accretion tracer Humphreys $\alpha$ H ${i}$ (7--6). The short-wavelength MIRI data resolve two continuum sources A and B, where mid-infrared source A is associated with the main mm continuum peak. The combination of mid-infrared and mm data reveals a young cluster in its making. Combining the mid-infrared outflow tracer $H_2$ , [ FeII ] and [ NeII ] with mm SiO data shows a complex interplay of at least three molecular outflows driven by protostars in the forming cluster. Furthermore, the Humphreys $\alpha$ line is detected at a 3--4 $\sigma$ level towards the mid-infrared sources A and B. Following [Rigliaco, Pascucci and Duchene (2015)]() , one can roughly estimate both accretion luminosities and corresponding accretion rates between $\sim$ 2.6 $\times 10^{-6}$ and $\sim$ 0.9 $\times 10^{-4}$ $M_{\odot}yr^{-1}$ . This is discussed in the context of the observed outflow rates. The analysis of the MIRI MRS observations for this young high-mass star-forming region reveals connected outflow and accretion signatures. Furthermore, they outline the enormous potential of JWST to boost our understanding of the physical and chemical processes during star formation.

</div>

<div id="div_fig1">

<img src="tmp_2303.13172/./outflows.png" alt="Fig8" width="100%"/>

**Figure 8. -** Outflow images in NOEMA SiO(2-1) (integrated emission, left panel), JWST [FeII]($^4F_{9/2}$--$^6D_{9/2}$)@5.34 $\mu$m (middle panel) and JWST $H_2$(0--0)S(7)@5.511 $\mu$m (right panel). The red and blue contours show the red- and blue-shifted NOEMA SiO(2--1) emission in the labeled velocity regimes ($v_{\rm lsr}=-50.2$ km s$^{-1}$). Contour levels are from 30 to 90\%(step 10\%) of the respective integrated peak emission. The magenta contours show the 5.2 $\mu$m continuum emission (steps 20 to 90\% by 15\%). The green contours in the left panel outline the 1.3 mm continuum starting at the 20$\sigma$ levels (1$\sigma$=1.13 mJy beam$^{-1}$). A linear scale bar is presented in the middle panel, and the green lines in the right panel outline and label potential outflows. The two mid-infrared sources are labeled in the right panel as well. The dashed line in the left panel marks a potentially precessing outflow corresponding to outflow (I). A green star in all panels marks the main mm peak position labeled mmA1 in [Cesaroni, Beuther and Ahmadi (2019)](). North and west are labeled in the right panel. The resolution elements are shown at the bottom right of each panel, left SiO(2--1) in grey and 1.3 mm continuum as line, middle and right [FeII] and $H_2$, respectively. (*outflows*)

</div>
<div id="div_fig2">

<img src="tmp_2303.13172/./outflows.png" alt="Fig15" width="100%"/>

**Figure 15. -** Outflow images in NOEMA SiO(2-1) (integrated emission, left panel), JWST [FeII]($^4F_{9/2}$--$^6D_{9/2}$)@5.34 $\mu$m (middle panel) and JWST $H_2$(0--0)S(7)@5.511 $\mu$m (right panel). The red and blue contours show the red- and blue-shifted NOEMA SiO(2--1) emission in the labeled velocity regimes ($v_{\rm lsr}=-50.2$ km s$^{-1}$). Contour levels are from 30 to 90\%(step 10\%) of the respective integrated peak emission. The magenta contours show the 5.2 $\mu$m continuum emission (steps 20 to 90\% by 15\%). The green contours in the left panel outline the 1.3 mm continuum starting at the 20$\sigma$ levels (1$\sigma$=1.13 mJy beam$^{-1}$). A linear scale bar is presented in the middle panel, and the green lines in the right panel outline and label potential outflows. The two mid-infrared sources are labeled in the right panel as well. The dashed line in the left panel marks a potentially precessing outflow corresponding to outflow (I). A green star in all panels marks the main mm peak position labeled mmA1 in [Cesaroni, Beuther and Ahmadi (2019)](). North and west are labeled in the right panel. The resolution elements are shown at the bottom right of each panel, left SiO(2--1) in grey and 1.3 mm continuum as line, middle and right [FeII] and $H_2$, respectively. (*outflows*)

</div>
<div id="div_fig3">

<img src="tmp_2303.13172/./23385_overview2.png" alt="Fig5" width="100%"/>

**Figure 5. -** Overview of the region around IRAS 23385+6053. The left panel shows a large-scale overview while the middle and right panels present  zoom-ins. The color-scale in the left and middle panels show the 12 $\mu$m emission from WISE at an angular resolution of $6.5"$ ([Wright, Eisenhardt and Mainzer 2010]()) . The little squares outline the field of view of the MIRI IFU mosaic in channel 1 (each square $3.7"\times 3.7"$). The color-scale in the right panel shows the new MIRI 5.2 $\mu$m continuum emission (see also Fig. \ref{continuum}). The contours in the middle and right panels present the 1.3 mm continuum emission  ([Beuther, Mottram and Ahmadi 2018](), [Cesaroni, Beuther and Ahmadi 2019]()) . The contours start at the 5$\sigma$ level of 0.55 mJy beam$^{-1}$ and continue in 25$\sigma$ steps. The black and red/blue lines in the middle panel show the directions of outflows identified in [Molinari, et. al (1998)]() and [Cesaroni, Beuther and Ahmadi (2019)](), respectively. The right panel marks the mid-infrared sources A and B as well as the mm sources labeled "mm" with the corresponding source letters from [Cesaroni, Beuther and Ahmadi (2019)](). Mid-infrared source A and mm source mmA2 are spatially co-located. A linear scale-bar is shown in all panels. (*overview*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2303.13172"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

172  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

3  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
